<div style="display:flex; gap:20px;">
  <div style="background-color:'#1E1E1E'; padding:10px;">
    <img src="output/visuals_Re200.gif" width="600">
  </div>
  <div style="background-color:'#1E1E1E'; padding:10px;">
    <img src="output/visuals_Re400.gif" width="600">
  </div>
</div>

### Import core libraries
We start by importing PyTorch and other essential libraries.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from torch.optim.lr_scheduler import ReduceLROnPlateau

# for cleaner look (set your theme to dark mode)
plt.style.use('dark_background')
plt.rcParams['figure.facecolor'] = '#1E1E1E'
plt.rcParams['axes.facecolor'] = '#1E1E1E'
plt.rcParams['savefig.facecolor'] = '#1E1E1E'

from tqdm import tqdm
from IPython.display import Image

In [ ]:
################# quickfix for Snellius GPU MIG usage
import os 
import subprocess
if "MIG" in subprocess.check_output(["nvidia-smi", "-L"], text=True):
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"

### Check CUDA availability
Here we check if a GPU is available for faster training. GPU usage is highly recommended.

In [ ]:
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA device name: {torch.cuda.get_device_name()}")

### Data preparation step
Fuse the raw data files so they can be read by the dataset object.

In [ ]:
# this box fuses the loose data files such that they can be read by the dataset object
base_in = '../../data/CFD/grid/loose/'
base_out2 = '../../data/CFD/grid/concat/'
os.makedirs(base_out2, exist_ok=True)
for Re in [100,150,200,250,300,350,400]:
    u = np.load(f"{base_in}u_grid_Re{Re}.npy")
    v = np.load(f"{base_in}v_grid_Re{Re}.npy")
    p = np.load(f"{base_in}p_grid_Re{Re}.npy")
    concat = np.stack([u, v, p], axis=1)
    filename_save = f"{base_out2}uvp_grid_Re{Re}.npy"
    np.save(filename_save, concat)

## Dataset object for standard ARM model

### Model input and output
- **Input**: fluid field information (e.g., velocity and pressure at the current timestep, possibly with coordinates/masks).  
- **Output**: predicted velocity and pressure fields at the **next timestep**.  
This setup makes the model autoregressive: predictions from one step are fed as input to predict the next.

### Define Dataset class
This class handles how fluid data is loaded and accessed. The dataset consists of 7 different trajectories of flow passing a cilindrical obstacle, each with their own Reynolds number. For the training data we use $Re$=[100,200,300,400] and for validation we use zero-shot validation on $Re$=[150,250,350]. The data consists of the $V_x$, $V_y$ and $p$ fields in 2 spatial dimensions and time. At the top of the notebook two example trajectories can be seen. The dataset object also supplies an obstacle mask, the coordinates and the option to flip-augment the data. Only the boolean obstacle mask will be used.

In [ ]:
class FlowDataset(Dataset):
    def __init__(self, filenames, flip_augmentation=False, timesample=1):
        self.sequences = []
        self.index_map = []
        self.flip_augmentation = flip_augmentation

        # coordinates
        self.coordsy = np.linspace(-5, 5, 64, endpoint=True)
        self.coordsx = np.linspace(-10, 10, 128, endpoint=True)
        self.coords = np.array(np.meshgrid(self.coordsx, self.coordsy)).T.reshape(128, 64, 2)
        self.coords = torch.tensor(self.coords, dtype=torch.float32).permute(2, 1, 0).cuda()

        # use coordinates to make obstacle mask
        center = torch.tensor([-5.0, 0.0], device=self.coords.device).view(2, 1, 1)
        radius = 0.5
        squared_distance = ((self.coords - center) ** 2).sum(dim=0) 
        self.mask = squared_distance < radius**2  # shape [64, 128]
        self.mask = self.mask.unsqueeze(0).cuda()

        # sample/read the data
        for seq_idx, filename in enumerate(filenames):
            data = np.load(filename)  
            data = data[::timesample] 
            self.sequences.append(data)
            T = data.shape[0]
            self.index_map.extend([(seq_idx, t) for t in range(T - 1)])

    def __len__(self):
        return len(self.index_map)

    def __getitem__(self, idx):
        seq_idx, t = self.index_map[idx]
        seq = self.sequences[seq_idx]
        input = seq[t]    
        target = seq[t + 1] 

        # if flip augmentation is true then flip the data horizontally 50% of the time
        if self.flip_augmentation and np.random.rand() > 0.5:
            input = self.flip(input)
            target = self.flip(target)
        return (
                self.mask, 
                self.coords, 
                torch.tensor(input, dtype=torch.float32), 
                torch.tensor(target, dtype=torch.float32)
                )
        
    def get_trajectory(self, seq_idx):
        # returns full trajectory
        seq = self.sequences[seq_idx]
        return (
            self.mask.unsqueeze(0), 
            self.coords.unsqueeze(0), 
            torch.tensor(seq, dtype=torch.float32)
        )

    def flip(self, x):
        x = np.flip(x, axis=2).copy()
        x[1] *= -1
        return x

datafolder = '../../data/CFD/grid/concat/'
train_files = [
    'uvp_grid_Re100.npy',
    'uvp_grid_Re200.npy',
    'uvp_grid_Re300.npy',
    'uvp_grid_Re400.npy'
]
val_files = [
    'uvp_grid_Re150.npy',
    'uvp_grid_Re250.npy',
    'uvp_grid_Re350.npy'
]
train_files = [datafolder + f for f in train_files]
val_files = [datafolder + f for f in val_files]

dt = 20 # only sample every dt timesteps
batch_size = 64
train_dataset = FlowDataset(train_files, flip_augmentation=False, timesample=dt)
val_dataset = FlowDataset(val_files, flip_augmentation=False, timesample=dt)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

## Model

### Define Model architecture
Here we build the building blocks of the autoregressive model.

### U-Net architecture (overview)
The model is based on a **U-Net**, which is a type of encoder–decoder neural network with skip connections.
- **Encoder**: progressively downsamples the input, extracting features at multiple scales.
- **Decoder**: upsamples the features back to the original resolution.
- **Skip connections**: link encoder and decoder layers at the same scale, helping the model preserve spatial details.

Below is a generic diagram of U-Net (not exactly your model, but the same idea):



In [ ]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels, dropprob=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, padding_mode='replicate'),
            nn.BatchNorm2d(out_channels),
            nn.GELU(),

            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, padding_mode='replicate'),
            nn.BatchNorm2d(out_channels),
            nn.GELU(),
            nn.Dropout2d(p=dropprob)
        )

    def forward(self, x):
        return self.net(x)

class SimpleUNet(nn.Module):
    def __init__(self, in_channels, out_channels, base_channels=64,mult=[1, 2, 4, 8]):
        super().__init__()

        # Encoder
        self.enc1 = DoubleConv(in_channels, base_channels * mult[0])
        self.pool1 = nn.MaxPool2d(2)
        self.enc2 = DoubleConv(base_channels * mult[0], base_channels * mult[1])
        self.pool2 = nn.MaxPool2d(2)
        self.enc3 = DoubleConv(base_channels * mult[1], base_channels * mult[2])
        self.pool3 = nn.MaxPool2d(2)

        # Bottleneck
        self.bottleneck = DoubleConv(base_channels * mult[2], base_channels * mult[3])

        # Decoder
        self.up3 = nn.ConvTranspose2d(base_channels * mult[3], base_channels * mult[2], kernel_size=2, stride=2)
        self.dec3 = DoubleConv(2 *base_channels * mult[2], base_channels * mult[2])
        self.up2 = nn.ConvTranspose2d(base_channels * mult[2], base_channels * mult[1], kernel_size=2, stride=2)
        self.dec2 = DoubleConv(2 * base_channels * mult[1], base_channels * mult[1])
        self.up1 = nn.ConvTranspose2d(base_channels * mult[1], base_channels * mult[0], kernel_size=2, stride=2)
        self.dec1 = DoubleConv(2 * base_channels * mult[0], base_channels * mult[0])
        
        self.out_conv = nn.Conv2d(base_channels * mult[0], out_channels, kernel_size=1)

    def forward(self, x):

        x1 = self.enc1(x)
        x2 = self.enc2(self.pool1(x1))
        x3 = self.enc3(self.pool2(x2))

        x4 = self.bottleneck(self.pool3(x3))

        x = self.up3(x4)
        x = torch.cat([x, x3], dim=1)
        x = self.dec3(x)
        x = self.up2(x3)
        x = torch.cat([x, x2], dim=1)
        x = self.dec2(x)
        x = self.up1(x)
        x = torch.cat([x, x1], dim=1)
        x = self.dec1(x)

        return self.out_conv(x)
    
def _amount_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def save_model(model, path):
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")

In [ ]:
base_channels = 64
use_coords = False
use_geometry = True
in_channels = 3
if use_coords:
    in_channels += 2 
if use_geometry:
    in_channels += 1  

epochs = 400
init_lr = 0.001
factor = 0.1
patience = 20
min_lr = 1e-7

train_mode = False # determines whether the model will be trained or loaded from file
save_mode = False # if train_mode is True, then saves the model is save_mode is True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleUNet(in_channels=in_channels, out_channels=3, base_channels=base_channels, mult=[1, 2, 2, 2]).to(device)
print(f"Amount of parameters in model: {_amount_params(model)}, will use {in_channels} input channels")
# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=init_lr)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=factor, patience=patience, min_lr=min_lr)

In [ ]:
def _input_fuse(mask, coords, input, use_coords, use_geometry):
    # this function fuses the input channels, obstacle mask coordinates and the velocity and pressure data
    if use_coords and use_geometry:
        input = torch.cat([mask, coords, input], dim=1)
    elif use_coords and not use_geometry:
        input = torch.cat([coords, input], dim=1)
    elif not use_coords and use_geometry:
        input = torch.cat([mask, input], dim=1)
    return input

### Training function
Define how the model will be trained (loop, loss, optimizer, etc.). Currently we load the model from memory instead of training.

In [ ]:
def train(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs):

    for epoch in range(epochs):
        train_losses = []
        val_loss = []

        model.train()
        train_bar = tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{epochs}", leave=False) # for cleaner look during training
        for mask, coords, input, target in train_bar:
            mask, coords, input, target = mask.to(device), coords.to(device), input.to(device), target.to(device)
            optimizer.zero_grad()
            input = _input_fuse(mask, coords, input, use_coords, use_geometry) #concat input channels
            output = model(input)
            train_loss = criterion(output, target)
            train_loss.backward()
            optimizer.step()
            train_losses.append(train_loss.item())
            train_bar.set_postfix(loss=f"{train_loss.item():.6f}")

        model.eval()
        val_bar = tqdm(val_loader, desc=f"Validation Epoch {epoch+1}/{epochs}", leave=False)
        with torch.no_grad():
            for mask, coords, input, target in val_bar:
                mask, coords, input, target = mask.to(device), coords.to(device), input.to(device), target.to(device)
                input = _input_fuse(mask, coords, input, use_coords, use_geometry)
                output = model(input)
                loss = criterion(output, target).item()
                val_loss.append(loss)
                val_bar.set_postfix(loss=f"{loss:.6f}")

        if epoch % 10 == 0 or epoch == epochs - 1: # plot every 10 epochs and also at the last epoch
            tqdm.write(f"Epoch {epoch+1}/{epochs} - Train Loss: {np.mean(train_losses):.6f}, Val Loss: {np.mean(val_loss):.6f}, LR: {optimizer.param_groups[0]['lr']:.8f}")
        scheduler.step(np.mean(val_loss)) 


if train_mode == True:
    print("Training mode enabled, starting training...")
    train(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs)
    if save_mode:
        save_model(model, '../../models/CFD/model1.pth')
else:
    print("Loading model from file...")
    model.load_state_dict(torch.load('../../models/CFD/model1.pth'))

torch.cuda.empty_cache() # clear cache for memory management

### Rollout method
A **rollout** means running the trained model forward in time for many steps, using its predictions as inputs for the next steps.  
This is how we evaluate the model’s ability to simulate fluid dynamics beyond a single timestep.

The following block also contains helper functions for creating animations and plotting comparisons...

In [ ]:
# helper functions

def rollout(model, mask, coords, init_frame, use_coords, use_geometry, length=100):
    # this function does a rollout of the ARM
    model.eval()
    with torch.no_grad():
        input = _input_fuse(mask, coords, init_frame, use_coords, use_geometry)
        output_sequence = []
        output_sequence.append(init_frame.squeeze(0).cpu())

        for _ in range(length):
            output = model(input)
            output_sequence.append(output.squeeze(0).cpu())
            input = _input_fuse(mask, coords, output, use_coords, use_geometry)

    return torch.stack(output_sequence)

def magnitude(tensor):
    # calculates the radial componentn/magnitude of the 2D velocity field
    return torch.sqrt(tensor[:,0,:,:]**2 + tensor[:,1,:,:]**2)

def animate_rollout(stacked_pred_vel, stacked_true_vel, stacked_pred_p, stacked_true_p, output_path="output/rollout.gif"):
    # this function makes the animation file, given the predicted and true velocity and pressure fields
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    timesteps, x_dim, y_dim = stacked_pred_vel.shape
    if timesteps > stacked_true_vel.shape[0]:
        timesteps = stacked_true_vel.shape[0]
    stacked_pred_vel, stacked_true_vel = stacked_pred_vel.squeeze(1).cpu().numpy(), stacked_true_vel.squeeze(1).cpu().numpy()
    stacked_pred_p, stacked_true_p = stacked_pred_p.squeeze(1).cpu().numpy(), stacked_true_p.squeeze(1).cpu().numpy()
    vmax_vel = min(np.max(stacked_pred_vel), np.max(stacked_true_vel))
    vmax_p = min(np.max(stacked_pred_p), np.max(stacked_true_p))
    fig, axes = plt.subplots(2, 3, figsize=(15, 5))
    titles = ["Pred", "True", "Diff"]
    imgs = []

    for i in range(2):
        row_imgs = []
        for j in range(3):
            ax = axes[i, j]
            img = ax.imshow(np.zeros((x_dim, y_dim)), cmap='viridis', vmin=0, vmax=vmax_vel if i == 0 else vmax_p)
            ax.set_title(titles[j])
            ax.set_xticks([])
            ax.set_yticks([])
            for spine in ax.spines.values():
                spine.set_visible(False)
            row_imgs.append(img)
        imgs.append(row_imgs)

    axes[0,0].set_ylabel(r'$\sqrt{v_x^2 + v_y^2}$')
    axes[1,0].set_ylabel(r'$p$')
    imgs[0][2].set_cmap('magma')
    imgs[1][2].set_cmap('magma')
    
    def init():
        imgs[0][0].set_data(stacked_pred_vel[0])
        imgs[0][1].set_data(stacked_true_vel[0])
        imgs[1][0].set_data(stacked_pred_p[0])
        imgs[1][1].set_data(stacked_true_p[0])
        return imgs

    def update(frame):

        imgs[0][0].set_data(stacked_pred_vel[frame])
        imgs[0][1].set_data(stacked_true_vel[frame])
        imgs[1][0].set_data(stacked_pred_p[frame])
        imgs[1][1].set_data(stacked_true_p[frame])
        imgs[0][2].set_data(np.abs(stacked_true_vel[frame] - stacked_pred_vel[frame]))
        imgs[1][2].set_data(np.abs(stacked_true_p[frame] - stacked_pred_p[frame]))

        fig.suptitle(f"timestep {frame + 1}\n"
                     #f"vmin: {vmin_frame:.4f}, vmax: {vmax_frame:.4f}")
                        #f"vmax: {vmax_frame:.3f}"
                        )
        return imgs

    ani = animation.FuncAnimation(fig, update, frames=timesteps, init_func=init, blit=False, interval=150)
    ani.save(output_path, writer="ffmpeg")
    plt.close()

def _generate_sequence(model, val_dataset, starting_timestep, length, trajectory_select=1):
    # this function manages the rollout, magnitude calculation and gets the data ready for plotting functionality or animation functionality
    model.eval()
    with torch.no_grad():
        mask, coords, val_traj = val_dataset.get_trajectory(trajectory_select)
        mask, coords, val_traj = mask.to(device), coords.to(device), val_traj.to(device)
        val_traj = val_traj[starting_timestep:]

        anim_data = rollout(model, mask, coords, val_traj[0].unsqueeze(0), use_coords, use_geometry, length=41) 
        velocity = magnitude(anim_data).cpu()
        velocity_test = magnitude(val_traj).cpu()
        pressure = anim_data[:,2,:,:].cpu()
        pressure_test = val_traj[:,2,:,:].cpu()
    return velocity, velocity_test, pressure, pressure_test

def plot_comparison(velocity, velocity_test, pressure, pressure_test):
    # this function plots the comparison between predicted and true fields at some timesteps
    fig, ax = plt.subplots(4, 4, figsize=(10, 7))
    timesteps = [0, 1, 5, 10]
    for i, t in enumerate(timesteps):
        # turn axis off
        for j in range(4):
            # only remove axis ticks
            ax[i, j].set_xticks([])
            ax[i, j].set_yticks([])
            for spine in ax[i, j].spines.values():
                spine.set_visible(False)
        # plot the velocity magnitude and pressure at each time step
        ax[i, 0].imshow(velocity[t].numpy(), cmap='viridis')
        ax[i, 2].imshow(pressure[t].numpy(), cmap='viridis')
        ax[i, 1].imshow(velocity_test[t].numpy(), cmap='viridis')
        ax[i, 3].imshow(pressure_test[t].numpy(), cmap='viridis')
    ax[0, 0].set_title('Predicted Velocity Magnitude')
    ax[0, 2].set_title('Predicted Pressure')
    ax[0, 1].set_title('True Velocity Magnitude')
    ax[0, 3].set_title('True Pressure')
    ax[0, 0].set_ylabel('t={t}'.format(t=timesteps[0]))
    ax[1, 0].set_ylabel('t={t}'.format(t=timesteps[1]))
    ax[2, 0].set_ylabel('t={t}'.format(t=timesteps[2]))
    ax[3, 0].set_ylabel('t={t}'.format(t=timesteps[3]))
    plt.tight_layout()
    plt.show()


### Set starting timestep
Choose the timestep where the prediction sequence begins and call the animation function of the model.

In [ ]:
starting_timestep = 6
velocity, velocity_true, pressure, pressure_true = _generate_sequence(model, val_dataset, starting_timestep, length=41, trajectory_select=1)
animate_rollout(velocity, velocity_true, pressure, pressure_true, output_path="output/rollout_model1.gif")
Image(filename="output/rollout_model1.gif")

### Plot comparison
Here we visualize predicted vs true velocity and pressure fields side-by-side. Although the model predicts the next timestep quite well, the predictions deviate from the true trajectories in the long run...

In [ ]:
plot_comparison(velocity, velocity_true, pressure, pressure_true)

## Push-forward training


Push-forward training means that during training, the model is rolled out over several timesteps by feeding its **own predictions** back as input.  
- At step 1: use ground truth input → predict next step.  
- At step 2: feed the prediction from step 1 → predict next step.  
- Continue for multiple steps.  

This helps the model learn to stay stable during long rollouts, where small errors can otherwise accumulate quickly.

In [ ]:
# since we are now planning on doing the pushforward trick, the
# data pipeline needs to be changed such that it supplements multiple timesteps as target from which a single target can be selected

class FlowDatasetPushForward(Dataset):
    def __init__(self, filenames, flip_augmentation=False, timesample=1, forward_steps=5):
        self.sequences = []
        self.index_map = []
        self.flip_augmentation = flip_augmentation
        self.forward_steps = forward_steps # this value determines how many forward timesteps are returned by the dataloader

        # coordinates
        self.coordsy = np.linspace(-5, 5, 64, endpoint=True) # coords are hardcoded
        self.coordsx = np.linspace(-10, 10, 128, endpoint=True)
        self.coords = np.array(np.meshgrid(self.coordsx, self.coordsy)).T.reshape(128, 64, 2)
        self.coords = torch.tensor(self.coords, dtype=torch.float32).permute(2, 1, 0).cuda()

        # obstacle mask 
        center = torch.tensor([-5.0, 0.0], device=self.coords.device).view(2, 1, 1)  
        radius = 0.5
        squared_distance = ((self.coords - center) ** 2).sum(dim=0)
        self.mask = (squared_distance < radius ** 2).unsqueeze(0).cuda()

        # Load/sample data
        for seq_idx, filename in enumerate(filenames):
            data = np.load(filename)   # [T, C, H, W]
            data = data[::timesample]
            self.sequences.append(data)
            T = data.shape[0]
            # ensure enough room for multiple forward solves/rollout steps
            self.index_map.extend([(seq_idx, t) for t in range(T - forward_steps)])

    def __len__(self):
        return len(self.index_map)

    def __getitem__(self, idx):
        seq_idx, t = self.index_map[idx]
        seq = self.sequences[seq_idx]

        input = seq[t]                               # [C, H, W]
        targets = seq[t+1:t+self.forward_steps+1]     # [forward_steps, C, H, W]

        if self.flip_augmentation and np.random.rand() > 0.5: # flip-augment if wanted
            input = self.flip(input)
            targets = [self.flip(target) for target in targets]

        return (
            self.mask, 
            self.coords, 
            torch.tensor(input, dtype=torch.float32), 
            torch.tensor(targets, dtype=torch.float32)
        )

    def get_trajectory(self, seq_idx):
        # returns full trajectory 
        seq = self.sequences[seq_idx]
        return (
            self.mask.unsqueeze(0), 
            self.coords.unsqueeze(0), 
            torch.tensor(seq, dtype=torch.float32)
        )

    def flip(self, x):
        x = np.flip(x, axis=1).copy()
        x[2] *= -1
        return x

    
datafolder = '../../data/CFD/grid/concat/'
train_files = [
    'uvp_grid_Re100.npy',
    'uvp_grid_Re200.npy',
    'uvp_grid_Re300.npy',
    'uvp_grid_Re400.npy'
]
val_files = [
    'uvp_grid_Re150.npy',
    'uvp_grid_Re250.npy',
    'uvp_grid_Re350.npy'
]
train_files = [datafolder + f for f in train_files]
val_files = [datafolder + f for f in val_files]

dt = 20 # only sample from data every dt timesteps
batch_size = 32
max_forward_steps = 4 # maximum number of push-forward steps
train_dataset = FlowDatasetPushForward(train_files, flip_augmentation=False, timesample=dt, forward_steps=max_forward_steps)
val_dataset = FlowDatasetPushForward(val_files, flip_augmentation=False, timesample=dt, forward_steps=max_forward_steps)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

model3 = SimpleUNet(in_channels=in_channels, out_channels=3, base_channels=base_channels, mult=[1, 2, 4, 4]).to(device)
optimizer3 = Adam(model3.parameters(), lr=init_lr)
scheduler3 = ReduceLROnPlateau(optimizer3, mode='min', factor=factor, patience=patience, min_lr=min_lr)
criterion = nn.MSELoss()

In [ ]:
# just to get a feeling for the data

for i, (mask, coords, inputs, targets) in enumerate(val_loader):
    print(f"  Mask shape: {mask.shape}")
    print(f"  Inputs shape: {inputs.shape}")
    print(f"  Targets shape: {targets.shape}")
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(inputs[0,0].cpu().numpy(), cmap='viridis')
    ax[0].set_title("Input")
    ax[1].imshow(targets[0, max_forward_steps-1,0].cpu().numpy(), cmap='viridis')
    ax[1].set_title(f"{max_forward_steps} timesteps later")
    plt.show()
    break

Your job is to implement the push-forward training function. Use the _pushforward_schedule() function to sample the amount of forward rollout steps that your model needs to do during each training step. 
* Think about whether your model should backpropagate loss of all the rollout steps or only a specific portion
* Think about a suitable learning strategy that balances rollout-stability and single step accuracy (Your model might forget to accurately predict next-timesteps if only trained on the far-future).
* You are allowed to use the obstacle mask, but not the coordinates.

In [ ]:
def _pushforward_schedule(epochs, max_forward_steps=5, method="linear"):
    """
    provide a full pushforward schedule for the given number of epochs, based on the provided method
    :param epochs:              The number of epochs
    :param max_forward_steps:   The maximum number of forward steps
    :param method:              The method to use for scheduling. Options = [linear, exponential, random, cyclic, constant, warmup]
    :return:                    numpy array of length epochs with values in range [1, max_forward_steps]
    """
    match method:
        case "linear":
            # Linearly increase number of forward steps across epochs until maximum
            steps = np.linspace(1, max_forward_steps, epochs)
            return np.rint(steps).astype(np.int32)
        case "exponential":
            # Exponentially increase number of forward steps
            steps = np.linspace(0, 1, epochs)
            return np.clip((max_forward_steps * (steps**2)).astype(np.int32), 1, max_forward_steps)
        case "random":
            # Uniformly random draw the number of forward steps
            return np.random.randint(1, max_forward_steps + 1, size=epochs)
        case "cyclic":
            # Cycle over the possible number of forward steps at a constant rate
            cycle = np.concatenate([np.arange(1, max_forward_steps + 1),
                                    np.arange(max_forward_steps-1, 0, -1)])
            return np.resize(cycle, epochs).astype(np.int32)
        case "constant":
            # Always do max number of steps
            return np.full(epochs, max_forward_steps).astype(np.int32)
        case "warmup":
            # Build up to constant with a series of warmup epochs
            warmup_epochs = epochs // 3
            ramp = np.linspace(1, max_forward_steps, warmup_epochs)
            rest = np.full(epochs - warmup_epochs, max_forward_steps)
            return np.concatenate([ramp, rest]).astype(np.int32)
        case _:
            raise NotImplementedError("Provided method not implemented")



def train_PF(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs,
             max_forward_steps=5, single_step_fraction=0.15, backpropagate_until=1):
    pushforward_schedule = _pushforward_schedule(epochs, max_forward_steps=max_forward_steps, method="cyclic")
    for epoch in range(epochs):
        train_losses = []
        val_loss = []
        steps = pushforward_schedule[epoch] - 1  # We always do 1 step

        model.train()
        train_bar = tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{epochs}", leave=False) # for cleaner look during training
        for mask, coords, input, target in train_bar:
            mask, coords, input, target = mask.to(device), coords.to(device), input.to(device), target.to(device)
            target = target[:,:steps + 1,:,:,:] # Trim targets we won't use
            optimizer.zero_grad()
            new_input = _input_fuse(mask, coords, input, use_coords, use_geometry) #concat input channels
            output = input + model(new_input)
            if np.random.rand() < single_step_fraction:
                target = target[:,0,:,:,:]
            else:
                for step in range(steps):
                    new_input = _input_fuse(mask, coords, output.detach(), use_coords, use_geometry)
                    output = output + model(new_input)

                target = target[:,-1:,:,:,:].squeeze(1)

            if target.shape != input.shape and input.shape != output.shape:
                raise ValueError("Target, input, and output must have same shape")
            train_loss = criterion(output, target)
            train_loss.backward()
            optimizer.step()
            train_losses.append(train_loss.item())
            train_bar.set_postfix(loss=f"{train_loss.item():.6f}")

        model.eval()
        val_bar = tqdm(val_loader, desc=f"Validation Epoch {epoch+1}/{epochs}", leave=False)
        with torch.no_grad():
            for mask, coords, input, target in val_bar:
                mask, coords, input, target = mask.to(device), coords.to(device), input.to(device), target.to(device)
                target = target[:,:steps + 1,:,:,:] # Trim targets we won't use
                new_input = _input_fuse(mask, coords, input, use_coords, use_geometry)
                output = input + model(new_input)
                if np.random.rand() < single_step_fraction:
                    target = target[:,0,:,:,:]
                else:
                    for step in range(steps):
                        new_input = _input_fuse(mask, coords, output.detach(), use_coords, use_geometry)
                        output = output + model(new_input)

                    target = target[:,-1:,:,:,:].squeeze(1)
                if target.shape != input.shape and input.shape != output.shape:
                    raise ValueError("Target, input, and output must have same shape")
                loss = criterion(output, target).item()
                val_loss.append(loss)
                val_bar.set_postfix(loss=f"{loss:.6f}")

        if epoch % 10 == 0 or epoch == epochs - 1: # plot every 10 epochs and also at the last epoch
            tqdm.write(f"Epoch {epoch+1}/{epochs} - Train Loss: {np.mean(train_losses):.6f}, Val Loss: {np.mean(val_loss):.6f}, LR: {optimizer.param_groups[0]['lr']:.8f}")
        scheduler.step(np.mean(val_loss))

train_mode = True
save_mode = False

if train_mode == True:
    print("Training mode enabled, starting training...")
    train_PF(model3, train_loader, val_loader, criterion, optimizer3, scheduler3, epochs, max_forward_steps=max_forward_steps)
    print("Training loss to beat: 0.002151")
    if save_mode:
        save_model(model3, '../../models/CFD/model3.pth')
else:
    print("Loading model from file...")
    model3.load_state_dict(torch.load('../../models/CFD/model3.pth'))

torch.cuda.empty_cache()

Let us now look at the animation and plots from the push-forward trained model. How does the new model compare to the old model?

In [ ]:
def _generate_sequence_residual(model, val_dataset, starting_timestep, length, trajectory_select=1):
    # this function manages the rollout, magnitude calculation and gets the data ready for plotting functionality or animation functionality
    model.eval()
    with torch.no_grad():
        mask, coords, val_traj = val_dataset.get_trajectory(trajectory_select)
        mask, coords, val_traj = mask.to(device), coords.to(device), val_traj.to(device)
        val_traj = val_traj[starting_timestep:]

        anim_data = rollout_residual(model, mask, coords, val_traj[0].unsqueeze(0), use_coords, use_geometry, length=41)
        velocity = magnitude(anim_data).cpu()
        velocity_test = magnitude(val_traj).cpu()
        pressure = anim_data[:,2,:,:].cpu()
        pressure_test = val_traj[:,2,:,:].cpu()
    return velocity, velocity_test, pressure, pressure_test

def rollout_residual(model, mask, coords, init_frame, use_coords, use_geometry, length=100):
    # this function does a rollout of the ARM
    model.eval()
    with torch.no_grad():
        input = init_frame
        input_fused = _input_fuse(mask, coords, init_frame, use_coords, use_geometry)
        output_sequence = []
        output_sequence.append(init_frame.squeeze(0).cpu())

        for _ in range(length):
            output = model(input_fused) + input
            output_sequence.append(output.squeeze(0).cpu())
            input = output
            input_fused = _input_fuse(mask, coords, output, use_coords, use_geometry)

    return torch.stack(output_sequence)

velocity, velocity_true, pressure, pressure_true = _generate_sequence_residual(model3, val_dataset, starting_timestep, length=41, trajectory_select=1)
animate_rollout(velocity, velocity_true, pressure, pressure_true, output_path="output/rollout_model3.gif")
Image(filename="output/rollout_model3.gif")

In [ ]:
plot_comparison(velocity, velocity_true, pressure, pressure_true)